In [ ]:
!wget https://github.com/DeepQuestAI/Fire-Smoke-Dataset/releases/download/v1/FIRE-SMOKE-DATASET.zip
!unzip FIRE-SMOKE-DATASET.zip

# Removing train and test data for "Smoke" images
import shutil
shutil.rmtree('FIRE-SMOKE-DATASET/Test/Smoke')
shutil.rmtree('FIRE-SMOKE-DATASET/Train/Smoke')

In [ ]:
%pip install keras_preprocessing keras tensorflow scipy

In [4]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "FIRE-SMOKE-DATASET/Train"

training_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.15,horizontal_flip=True,fill_mode='nearest')

VALIDATION_DIR = "FIRE-SMOKE-DATASET/Test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(224,224),
	shuffle = True,
	class_mode='categorical',
  batch_size = 128
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(224,224),
	class_mode='categorical',
	shuffle = True,
  batch_size= 14
)

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

input_tensor = Input(shape=(224, 224, 3))

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

2023-05-18 08:13:13.002520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-18 08:13:13.002770: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-18 08:13:13.002831: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-18 08:13:13.002879: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-18 08:13:13.002926: W tensorflow/c

In [6]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss')<=0.1099 and logs.get('loss')<=0.1099):
      print('\n\n Reached The Destination!')
      self.model.stop_training = True
callbacks = myCallback()

history = model.fit(
    train_generator,
    steps_per_epoch = 14,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = 14,
    callbacks=[callbacks]
)

Epoch 1/10


2023-05-18 08:13:25.070906: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 389316608 exceeds 10% of free system memory.
2023-05-18 08:13:25.516451: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 265814016 exceeds 10% of free system memory.


 1/14 [=>............................] - ETA: 2:20 - loss: 0.7114 - acc: 0.5859

2023-05-18 08:13:29.200100: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 389316608 exceeds 10% of free system memory.
2023-05-18 08:13:29.633197: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 265814016 exceeds 10% of free system memory.


 2/14 [===>..........................] - ETA: 49s - loss: 0.6571 - acc: 0.7305 

2023-05-18 08:13:33.314366: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 389316608 exceeds 10% of free system memory.


14/14 [==============================] - 87s 6s/step - loss: 7.3443 - acc: 0.8131 - val_loss: 0.1318 - val_acc: 0.9490
Epoch 2/10
14/14 [==============================] - 84s 6s/step - loss: 0.2606 - acc: 0.9187 - val_loss: 0.0977 - val_acc: 0.9643
Epoch 3/10
14/14 [==============================] - 79s 6s/step - loss: 0.1975 - acc: 0.9258 - val_loss: 0.1007 - val_acc: 0.9592
Epoch 4/10
14/14 [==============================] - 83s 6s/step - loss: 0.1244 - acc: 0.9522 - val_loss: 0.2947 - val_acc: 0.8673
Epoch 5/10
14/14 [==============================] - 82s 6s/step - loss: 0.1478 - acc: 0.9480 - val_loss: 0.4111 - val_acc: 0.8418
Epoch 6/10
14/14 [==============================] - 84s 6s/step - loss: 0.2301 - acc: 0.9462 - val_loss: 0.2962 - val_acc: 0.8929
Epoch 7/10
14/14 [==============================] - 97s 7s/step - loss: 0.0816 - acc: 0.9713 - val_loss: 0.2162 - val_acc: 0.9388
Epoch 8/10
14/14 [==============================] - 97s 7s/step - loss: 0.0679 - acc: 0.9767 - val_lo

In [7]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])


class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss')<=0.1099 and logs.get('loss')<=0.1099):
      print('\n\n Reached The Destination!')
      self.model.stop_training = True
callbacks = myCallback()

history = model.fit(
    train_generator,
    steps_per_epoch = 14,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = 14,
    callbacks=[callbacks]
)
print(len(base_model.layers))

Epoch 1/10
14/14 [==============================] - 131s 9s/step - loss: 0.1344 - acc: 0.9653 - val_loss: 0.4466 - val_acc: 0.8827
Epoch 2/10
14/14 [==============================] - 119s 8s/step - loss: 0.0230 - acc: 0.9934 - val_loss: 0.3666 - val_acc: 0.9388
Epoch 3/10
14/14 [==============================] - 119s 8s/step - loss: 0.0648 - acc: 0.9761 - val_loss: 0.3040 - val_acc: 0.9490
Epoch 4/10
14/14 [==============================] - 122s 9s/step - loss: 0.0297 - acc: 0.9910 - val_loss: 0.1540 - val_acc: 0.9745
Epoch 5/10
 8/14 [================>.............] - ETA: 54s - loss: 0.0152 - acc: 0.9941 

In [ ]:
model.save('model_file.h5')